In [1]:
# Imports
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split

from nn_utils import create_model, one_hot_encode

2023-03-12 13:04:24.430235: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-12 13:04:24.519194: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-12 13:04:24.519209: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-03-12 13:04:24.965123: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [2]:
quizzes = np.zeros((1000000, 81), np.int32)
solutions = np.zeros((1000000, 81), np.int32)

for i, line in enumerate(open('sudoku.csv', 'r').read().splitlines()[1:]):
    quiz, solution = line.split(",")
    for j, q_s in enumerate(zip(quiz, solution)):
        q, s = q_s
        quizzes[i, j] = q
        solutions[i, j] = s

X = quizzes.reshape((-1, 9, 9))
Y = solutions.reshape((-1, 9, 9))

assert len(X) == len(Y)               # Check same number of X's and Y'x loaded

In [4]:
assert one_hot_encode(X[0]).shape == (81,9) # Check output shape of OHE puzzle

# Create a new Tensor to hold the One Hot Encoded puzzles
shape = one_hot_encode(X[0]).shape
X_ohe = np.ndarray((len(X), shape[0], shape[1]))
Y_ohe = np.ndarray((len(Y), shape[0], shape[1]))

for i in range(len(X)):
    try:
        X_ohe[i] = one_hot_encode(X[i])
        Y_ohe[i] = one_hot_encode(Y[i])
    except ValueError:
        print(f"{i} malformed")

14861 malformed
118801 malformed
147953 malformed
168906 malformed
228197 malformed
482410 malformed
760120 malformed
781414 malformed
865168 malformed
994456 malformed


In [5]:
# Training/Test split
X_train, X_test, Y_train, Y_test = train_test_split(X_ohe, Y_ohe, test_size=0.01, random_state=42)

print(f"Training X: {X_train.shape}")
print(f"Training Y: {Y_train.shape}")

Training X: (990000, 81, 9)
Training Y: (990000, 81, 9)


In [6]:
# Create model of the Neural Network
# Model based on this paper: https://cs230.stanford.edu/files_winter_2018/projects/6939771.pdf
model = create_model()
model.compile(loss='binary_crossentropy', 
              optimizer='adam', 
              metrics='binary_accuracy')
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 81, 9, 64)         640       
                                                                 
 conv2d_1 (Conv2D)           (None, 81, 9, 64)         36928     
                                                                 
 conv2d_2 (Conv2D)           (None, 81, 9, 64)         36928     
                                                                 
 conv2d_3 (Conv2D)           (None, 81, 9, 64)         36928     
                                                                 
 conv2d_4 (Conv2D)           (None, 81, 9, 64)         36928     
                                                                 
 conv2d_5 (Conv2D)           (None, 81, 9, 32)         18464     
                                                                 
 conv2d_6 (Conv2D)           (None, 81, 9, 32)         9

2023-03-12 13:06:08.224552: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-03-12 13:06:08.224569: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-03-12 13:06:08.224586: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (pop-os): /proc/driver/nvidia/version does not exist
2023-03-12 13:06:08.224746: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
# Fit model to training data
model.fit(X_train, Y_train, batch_size=2, epochs=1)

In [ ]:
# Evaluate model
model.evaluate(X_test, Y_test)

In [ ]:
tf.keras.models.save_model(model,"model/")